<a href="https://colab.research.google.com/github/alessandronascimento/pyLiBELa/blob/alex-works/GridNumba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Baixando pyLiBELa {display-mode: "form"}
%%capture
!pip3 install condacolab
import condacolab
condacolab.install()
! apt-get install openbabel libopenbabel-dev python-dev-is-python3 zlib1g-dev libeigen3-dev libgsl-dev libnlopt-cxx-dev libgsl-dev
! mamba install -c anaconda py-boost

In [ ]:
! git init
! git remote add origin https://github.com/alessandronascimento/pyLiBELa.git
! git pull origin alex-works

In [1]:
! mv pyLiBELa/src src
! rm -rf pyLiBELa
! mkdir -p obj
! rm -f Makefile*
!wget https://raw.githubusercontent.com/alessandronascimento/pyLiBELa/main/Colabs/Makefile
!make -j4 

mv: cannot stat 'pyLiBELa/src': No such file or directory
--2023-05-16 16:57:59--  https://raw.githubusercontent.com/alessandronascimento/pyLiBELa/main/Colabs/Makefile
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5606 (5.5K) [text/plain]
Saving to: ‘Makefile’

Makefile            100%[===================>]   5.47K  --.-KB/s    in 0s      

2023-05-16 16:58:00 (48.9 MB/s) - ‘Makefile’ saved [5606/5606]

g++ -fPIC -O3 -I/usr/local/include/python3.10 -I/usr/include/openbabel3 -I/usr/include/eigen3 -DBUILD=0 -DHAVE_EIGEN -fopenmp -Wunused-value src/pyPARSER.cpp -c -o obj/pyPARSER.o
g++ -fPIC -O3 -I/usr/local/include/python3.10 -I/usr/include/openbabel3 -I/usr/include/eigen3 -DBUILD=0 -DHAVE_EIGEN -fopenmp -Wunused-value src/pyMol2.cpp -c -o obj/pyMol2.o
g++ -

In [15]:
!rm -rf obj

In [9]:
#@title Importando bibliotecas {display-mode: "form"}

from pyPARSER import *
from pyMol2 import *
from pyWRITER import *
from pyGrid import *
from pyCOORD_MC import *
from pyFindHB import * 
from pyEnergy2 import *
from pyGaussian import *
from pyConformer import *
from pyRAND import *
from pyMcEntropy import *
from pySA import *
from pyOptimizer import *
from pyMC import *
from pyFullSearch import *
from pyDocker import *

from tqdm import tqdm
import numpy as np
from numba import jit
import pdb

In [4]:
Input = PARSER()
Input.output = "teste" #@param {type:"string"}
Input.scoring_function = 0 #@param {type:"integer"}
Input.use_grids = True #@param {type:"boolean"}
Input.grid_prefix = "McGrid" #@param {type:"string"}
Input.search_box_x = 10.0 #@param {type:"number"}
Input.search_box_y = 10.0 #@param {type:"number"}
Input.search_box_z = 10.0 #@param {type:"number"}
Input.grid_spacing = 3 #@param {type:"number"}
Input.x_dim = 9.0 #@param {type:"number"}
Input.y_dim = 9.0 #@param {type:"number"}
Input.z_dim = 9.0 #@param {type:"number"}
Input.write_grids = True;

# Para que funcione a funcão

Input.dielectric_model = "constant" #@param {type: "string"}

In [6]:
Writer = WRITER(Input)
Coord = COORD_MC()
HB = FindHB()
lig = Mol2(Input, "test/lig.mol2.gz")
rec = Mol2(Input, "test/rec.mol2.gz")
Energy = Energy2(Input)

com = Coord.compute_com(lig)
grid = Grid(Input, Writer, rec, com)
refgrid = Grid(Input, Writer, rec, com)

In [ ]:
for i in range(len(rec.charges)):
  
  print(rec.charges[i], '\n')

In [ ]:
refgrid.compute_grid_softcore(rec)

In [ ]:
refgrid.compute_grid_softcore_HB_omp(rec)

In [ ]:
del(refgrid)

In [ ]:
del(grid)

In [ ]:
for i in range(grid.npointsy):
  print('\n')
  for j in range(grid.npointsz):
    print(f'{refgrid.rec_solv_gauss[0][i][j]:.2f}', end=' ')

In [ ]:
#@title compute_grid_softcore
def compute_grid_softcore(Rec, Grid, Input):
  
  # if (Input.scoring_function >= 2):
    #Grid.compute_grid_harcore(rec)
    #return

  
  for i in tqdm(range(Grid.npointsx)):
    x = (i*Grid.grid_spacing) + Grid.xbegin

    for j in range(Grid.npointsy):
      y = (j*Grid.grid_spacing) + Grid.ybegin

      for k in range(Grid.npointsz):
        z = (k*Grid.grid_spacing) + Grid.zbegin

        elec = 0
        vdwA = 0
        vdwB = 0
        solv = 0
        rec_solv = 0

        for n in range(Rec.N):
          d = Grid.distance(x, Rec.xyz[n][0], y, Rec.xyz[n][1], z, Rec.xyz[n][2])
          d2 = d**2
          d3 = d**3
          d6 = d2**3

          denom = (d3 + Input.deltaij_es3) ** (1/3)

          if (Input.dielectric_model == "constant"):
            elec += (332*Rec.charges[n]) / (Input.diel*denom)
            denom = (d6 + Input.deltaij_es6) ** (1/3)
            solv += ( ((Input.solvation_alpha * Rec.charges[n] * Rec.charges[n]) + 
                       Input.solvation_beta ) * np.exp((-denom/(2*Input.sigma**2))) / (Input.sigma**3) )
            rec_solv += (4/3) * np.pi * Rec.radii[n]**3 * np.exp((-denom/(2*Input.sigma**2))) / (Input.sigma**3)

          else:
            denom = (d6 + Input.deltaij_es6) ** (1/3)
            elec += (332 * Rec.charges[n]/(Input.diel*denom))
            solv += ( ((Input.solvation_alpha * Rec.charges[n] * Rec.charges[n]) + 
                       Input.solvation_beta ) * np.exp((-denom/(2*Input.sigma**2))) / (Input.sigma**3) )
            rec_solv += (4/3) * np.pi * Rec.radii[n]**3 * np.exp((-denom/(2*Input.sigma**2))) / (Input.sigma**3)

          denom = d6 + Input.deltaij6
          vdwA += (4096 * Rec.epsilons_sqrt[n] * Rec.radii[n]**6) / (denom**2)
          vdwB += (128 * Rec.epsilons_sqrt[n] * Rec.radii[n]**3) / (denom)

        # NOTE: assumo aqui que todos os arrays estão inicializados
        # e do tamanho correto
        Grid.elec_grid[i][j][k] = elec
        Grid.vdwA_grid[i][j][k] = vdwA
        Grid.vdwB_grid[i][j][k] = vdwB
        Grid.solv_gauss[i][j][k] = solv
        Grid.rec_solv_gauss[i][j][k] = rec_solv

  Grid.rec_si = 0
  for i in range(Rec.N):
    Grid.rec_si += (Input.solvation_alpha * Rec.charges[i] ** 2) + Input.solvation_beta



In [51]:
#@title compute_softcore_HB_omp


def compute_grid_softcore_HB_omp(rec, grid, input):
  
  #if input.scoring_function >= 2:
  #  compute_grid_hardcore_HB_omp(...)
  #  return

  HB_C12 = np.double(55332.873)
  HB_C10 = np.double(18393.199)

  for i in range(grid.npointsx):
    #pdb.set_trace()
    x = np.double(i*grid.grid_spacing + grid.xbegin) 

    for j in range(grid.npointsy):
      y = np.double(j*grid.grid_spacing + grid.ybegin) 

      for k in range(grid.npointsz):
        z = np.double(k*grid.grid_spacing + grid.zbegin) 

        elec = np.double(0.0)
        vdwA = np.double(0.0)
        vdwB = np.double(0.0) 
        solv = np.double(0.0) 
        rec_solv = np.double(0.0) 
        hb_donor = np.double(0.0) 
        hb_acceptor = np.double(0.0) 

        for a in range(rec.N):
          
          d2 = np.double(grid.distance_squared(x, rec.xyz[a][0], y, rec.xyz[a][1], z, rec.xyz[a][2]))
          d6 = np.double(np.power(d2, 3))
          
          denom = np.double(0.0)
            
          if input.dielectric_model == "constant":
            denom = np.power(d6 + input.deltaij_es6, 1/3)
            elec += (332.0*rec.charges[a])/(input.diel*denom)
            if i == 0 and j == 0 and k == 0:
              pass
              #print(f'({332*rec.charges[a]:.2f}, {input.diel*denom:.2f}, {(332.0*rec.charges[a])/(input.diel*denom):.2f}, {elec:.5f})\n')
            solv += ((input.solvation_alpha * rec.charges[a] * rec.charges[a]) + input.solvation_beta) * np.exp((-denom/(2 * np.power(input.sigma, 2)))) / (np.power(input.sigma, 3))
            rec_solv += (4/3) * np.pi * np.power(rec.radii[a], 3) * np.exp((-denom/(2*np.power(input.sigma, 2)))) / (np.power(input.sigma, 3))

          else:
            denom = np.power(d6 + input.deltaij_es6, 1/3)
            elec += (332.0*rec.charges[a]/(input.diel*denom))
            solv += ((input.solvation_alpha * np.power(rec.charges[a], 2)) + input.solvation_beta) * np.exp((-denom/(2 * np.power(input.sigma, 2)))) / (np.power(input.sigma, 3))
            rec_solv += (4/3) * np.pi * np.power(rec.radii[a], 3) * np.exp((-denom/(2*np.power(input.sigma, 2)))) / (np.power(input.sigma, 3))
          
          denom = (d6 + input.deltaij6)
          vdwA += (4096*rec.epsilons_sqrt[a] * np.power(rec.radii[a], 6)) / np.power(denom, 2)
          vdwB += ( 128*rec.epsilons_sqrt[a] * np.power(rec.radii[a], 3)) / denom

        for m in range(len(rec.HBdonors)):
          d2 = grid.distance_squared(rec.xyz[rec.HBdonors[m][1]][0], x, rec.xyz[rec.HBdonors[m][1]][1], y, rec.xyz[rec.HBdonors[m][1]][2], z)
          d10 = np.double(np.power(d2, 5))
          ang = np.double(grid.angle(rec.xyz[rec.HBdonors[m][0]][0], rec.xyz[rec.HBdonors[m][0]][1], rec.xyz[rec.HBdonors[m][0]][2], 
                          rec.xyz[rec.HBdonors[m][1]][0], rec.xyz[rec.HBdonors[m][1]][1], rec.xyz[rec.HBdonors[m][1]][2]))
          angle_term = np.double(np.power(np.cos(ang * np.pi / 180), 4))
          hb_donor += ((HB_C12/(d10*d2)) - (HB_C10/d10)) * angle_term

        for n in range(len(rec.HBacceptors)):
          d2 = grid.distance_squared(rec.xyz[rec.HBacceptors[n]][0], x, rec.xyz[rec.HBacceptors[n]][1], y, rec.xyz[rec.HBacceptors[n]][2], z)
          d10 = np.double(np.power(d2, 5))
          hb_acceptor += ((HB_C12/(d10*d2)) - (HB_C10/d10))

        grid.elec_grid[i][j][k] = elec
        grid.vdwA_grid[i][j][k] = vdwA
        grid.vdwB_grid[i][j][k] = vdwB
        grid.solv_gauss[i][j][k] = solv
        grid.rec_solv_gauss[i][j][k] = rec_solv
        grid.hb_donor_grid[i][j][k] = hb_donor
        grid.hb_acceptor_grid[i][j][k] = hb_acceptor;

  grid.rec_si = 0.0
  for a in range(rec.N):
    grid.rec_si += (input.solvation_alpha * np.power(rec.charges[a], 2)) + input.solvation_beta


In [64]:
#@title (NUMBA) compute_softcore_HB_omp

from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
import warnings

warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaPendingDeprecationWarning)

@jit
def compute_grid_softcore_HB_numba(rec, grid, input, grid_spacing, 
                                   npointsx, npointsy, npointsz):
  
  #if input.scoring_function >= 2:
  #  compute_grid_hardcore_HB_omp(...)
  #  return

  HB_C12 = np.double(55332.873)
  HB_C10 = np.double(18393.199)

  for i in range(grid.npointsx):
    #pdb.set_trace()
    x = np.double(i*grid.grid_spacing + grid.xbegin) 

    for j in range(grid.npointsy):
      y = np.double(j*grid.grid_spacing + grid.ybegin) 

      for k in range(grid.npointsz):
        z = np.double(k*grid.grid_spacing + grid.zbegin) 

        elec = np.double(0.0)
        vdwA = np.double(0.0)
        vdwB = np.double(0.0) 
        solv = np.double(0.0) 
        rec_solv = np.double(0.0) 
        hb_donor = np.double(0.0) 
        hb_acceptor = np.double(0.0) 

        for a in range(rec.N):
          
          d2 = np.double(grid.distance_squared(x, rec.xyz[a][0], y, rec.xyz[a][1], z, rec.xyz[a][2]))
          d6 = np.double(np.power(d2, 3))
          
          denom = np.double(0.0)
            
          if input.dielectric_model == "constant":
            denom = np.power(d6 + input.deltaij_es6, 1/3)
            elec += (332.0*rec.charges[a])/(input.diel*denom)
            if i == 0 and j == 0 and k == 0:
              pass
              #print(f'({332*rec.charges[a]:.2f}, {input.diel*denom:.2f}, {(332.0*rec.charges[a])/(input.diel*denom):.2f}, {elec:.5f})\n')
            solv += ((input.solvation_alpha * rec.charges[a] * rec.charges[a]) + input.solvation_beta) * np.exp((-denom/(2 * np.power(input.sigma, 2)))) / (np.power(input.sigma, 3))
            rec_solv += (4/3) * np.pi * np.power(rec.radii[a], 3) * np.exp((-denom/(2*np.power(input.sigma, 2)))) / (np.power(input.sigma, 3))

          else:
            denom = np.power(d6 + input.deltaij_es6, 1/3)
            elec += (332.0*rec.charges[a]/(input.diel*denom))
            solv += ((input.solvation_alpha * np.power(rec.charges[a], 2)) + input.solvation_beta) * np.exp((-denom/(2 * np.power(input.sigma, 2)))) / (np.power(input.sigma, 3))
            rec_solv += (4/3) * np.pi * np.power(rec.radii[a], 3) * np.exp((-denom/(2*np.power(input.sigma, 2)))) / (np.power(input.sigma, 3))
          
          denom = (d6 + input.deltaij6)
          vdwA += (4096*rec.epsilons_sqrt[a] * np.power(rec.radii[a], 6)) / np.power(denom, 2)
          vdwB += ( 128*rec.epsilons_sqrt[a] * np.power(rec.radii[a], 3)) / denom

        for m in range(len(rec.HBdonors)):
          d2 = grid.distance_squared(rec.xyz[rec.HBdonors[m][1]][0], x, rec.xyz[rec.HBdonors[m][1]][1], y, rec.xyz[rec.HBdonors[m][1]][2], z)
          d10 = np.double(np.power(d2, 5))
          ang = np.double(grid.angle(rec.xyz[rec.HBdonors[m][0]][0], rec.xyz[rec.HBdonors[m][0]][1], rec.xyz[rec.HBdonors[m][0]][2], 
                          rec.xyz[rec.HBdonors[m][1]][0], rec.xyz[rec.HBdonors[m][1]][1], rec.xyz[rec.HBdonors[m][1]][2]))
          angle_term = np.double(np.power(np.cos(ang * np.pi / 180), 4))
          hb_donor += ((HB_C12/(d10*d2)) - (HB_C10/d10)) * angle_term

        for n in range(len(rec.HBacceptors)):
          d2 = grid.distance_squared(rec.xyz[rec.HBacceptors[n]][0], x, rec.xyz[rec.HBacceptors[n]][1], y, rec.xyz[rec.HBacceptors[n]][2], z)
          d10 = np.double(np.power(d2, 5))
          hb_acceptor += ((HB_C12/(d10*d2)) - (HB_C10/d10))

        grid.elec_grid[i][j][k] = elec
        grid.vdwA_grid[i][j][k] = vdwA
        grid.vdwB_grid[i][j][k] = vdwB
        grid.solv_gauss[i][j][k] = solv
        grid.rec_solv_gauss[i][j][k] = rec_solv
        grid.hb_donor_grid[i][j][k] = hb_donor
        grid.hb_acceptor_grid[i][j][k] = hb_acceptor;

  #grid.rec_si = 0.0
  #for a in range(rec.N):
  #  grid.rec_si += (input.solvation_alpha * np.power(rec.charges[a], 2)) + input.solvation_beta


In [65]:
compute_grid_softcore_HB_numba(rec, grid, Input, grid.grid_spacing, grid.npointsx, grid.npointsy, grid.npointsz)

<ipython-input-64-e372264350a5>:9: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "compute_grid_softcore_HB_numba" failed type inference due to: non-precise type pyobject
During: typing of argument at <ipython-input-64-e372264350a5> (17)

File "<ipython-input-64-e372264350a5>", line 17:
def compute_grid_softcore_HB_numba(rec, grid, input, grid_spacing, 
    <source elided>

  HB_C12 = 55332.873
  ^

  @jit
<ipython-input-64-e372264350a5>:9: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "compute_grid_softcore_HB_numba" failed type inference due to: Cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "<ipython-input-64-e372264350a5>", line 20:
def compute_grid_softcore_HB_numba(rec, grid, input, grid_spacing, 
    <source elided>

  for i in range(grid.npointsx):
  ^

  @jit
/usr/local/lib/python3.10/dist-packages/numba/core/object_mode_passes.py:151: 

In [ ]:
#@title With write to file
_write_to_grid = False

with open("grid_output.txt", 'w') as file:
    for i in range(grid.npointsy):
      if _write_to_grid:
        file.write('\n')
      print('\n')
      for j in range(grid.npointsz):
        print(f'{grid.elec_grid[0][i][j]:.2f}', end = ' ')
        if _write_to_grid:
          file.write(f'{grid.elec_grid[0][i][j]:.2f}, ')

In [46]:
for i in range(grid.npointsx):
  print('\n------------------------')
  for j in range(grid.npointsy):
    print('\n')
    for k in range(grid.npointsz):
      print(f'{grid.elec_grid[i][j][k]:.2f}', end=' ')


------------------------


12.45 14.55 1.14 

8.42 3.97 3.52 

12.92 2.18 7.61 
------------------------


10.28 2.63 6.95 

10.29 7.07 6.27 

8.42 -0.19 5.66 
------------------------


12.65 4.65 11.27 

0.76 8.01 6.42 

6.35 4.13 1.62 

In [47]:
for i in range(grid.npointsx):
  print('\n------------------------')
  for j in range(grid.npointsy):
    print('\n')
    for k in range(grid.npointsz):
      print(f'{refgrid.elec_grid[i][j][k]:.2f}', end=' ')


------------------------


12.45 14.55 1.14 

8.42 3.97 3.52 

12.92 2.18 7.61 
------------------------


10.28 2.63 6.95 

10.29 7.07 6.27 

8.42 -0.19 5.66 
------------------------


12.65 4.65 11.27 

0.76 8.01 6.42 

6.35 4.13 1.62 

In [ ]:
refgrid.Input